In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
NETWORK = 'ConvNeXtBase'

METHOD = "STD-Max"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.849793,1.153479,1.712473,1.604126,1.037871,0.780016,0.686196,1.039208,0.773811,...,0.967454,0.615776,0.969945,1.031309,1.192442,0.876031,0.631890,1.399767,1.349431,1.315909
1,CHUM-002,0.796595,1.041349,1.375880,1.653947,1.311358,0.733445,0.632369,1.042104,0.793345,...,1.046854,0.634482,0.994855,0.972612,1.235414,0.870591,0.705144,1.445402,1.579015,1.587470
2,CHUM-006,0.730168,0.975537,1.358371,1.289311,1.214320,0.868420,0.744785,1.161157,1.063967,...,1.269750,0.874845,0.838466,0.987888,1.018574,0.760704,0.654079,0.951623,1.833445,1.313436
3,CHUM-007,0.748928,0.804768,1.377329,1.434227,1.105228,0.853569,0.671675,1.225464,1.025264,...,1.290362,0.917312,0.944272,0.974384,1.008991,0.777366,0.667562,1.167827,1.787023,1.551330
4,CHUM-008,0.858231,1.006652,1.674521,1.493922,1.257221,0.841633,0.774333,1.048915,1.028119,...,1.222317,0.809263,0.823017,0.913257,1.131655,0.829738,0.704352,0.999912,1.666580,1.590348


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.849793,1.153479,1.712473,1.604126,1.037871,0.780016,0.686196,1.039208,0.773811,...,0.967454,0.615776,0.969945,1.031309,1.192442,0.876031,0.631890,1.399767,1.349431,1.315909
1,CHUM-002,0.796595,1.041349,1.375880,1.653947,1.311358,0.733445,0.632369,1.042104,0.793345,...,1.046854,0.634482,0.994855,0.972612,1.235414,0.870591,0.705144,1.445402,1.579015,1.587470
2,CHUM-006,0.730168,0.975537,1.358371,1.289311,1.214320,0.868420,0.744785,1.161157,1.063967,...,1.269750,0.874845,0.838466,0.987888,1.018574,0.760704,0.654079,0.951623,1.833445,1.313436
3,CHUM-007,0.748928,0.804768,1.377329,1.434227,1.105228,0.853569,0.671675,1.225464,1.025264,...,1.290362,0.917312,0.944272,0.974384,1.008991,0.777366,0.667562,1.167827,1.787023,1.551330
4,CHUM-008,0.858231,1.006652,1.674521,1.493922,1.257221,0.841633,0.774333,1.048915,1.028119,...,1.222317,0.809263,0.823017,0.913257,1.131655,0.829738,0.704352,0.999912,1.666580,1.590348


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,0.849793,1.153479,1.712473,1.604126,1.037871,0.780016,0.686196,1.039208,0.773811,...,0.969945,1.031309,1.192442,0.876031,0.631890,1.399767,1.349431,1.315909,0,1704
1,CHUM-002,0.796595,1.041349,1.375880,1.653947,1.311358,0.733445,0.632369,1.042104,0.793345,...,0.994855,0.972612,1.235414,0.870591,0.705144,1.445402,1.579015,1.587470,1,439
2,CHUM-006,0.730168,0.975537,1.358371,1.289311,1.214320,0.868420,0.744785,1.161157,1.063967,...,0.838466,0.987888,1.018574,0.760704,0.654079,0.951623,1.833445,1.313436,0,1186
3,CHUM-007,0.748928,0.804768,1.377329,1.434227,1.105228,0.853569,0.671675,1.225464,1.025264,...,0.944272,0.974384,1.008991,0.777366,0.667562,1.167827,1.787023,1.551330,0,1702
4,CHUM-008,0.858231,1.006652,1.674521,1.493922,1.257221,0.841633,0.774333,1.048915,1.028119,...,0.823017,0.913257,1.131655,0.829738,0.704352,0.999912,1.666580,1.590348,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)